In [1]:
import os
import torch
from phi2_use_model_v1 import load_fine_tuned_model, generate_response

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


torch.cuda.empty_cache()
torch.cuda.is_available()
torch.cuda.current_device()
torch.cuda.get_device_name()
torch.cuda.memory_allocated()
torch.cuda.memory_reserved()
torch.cuda.memory_summary()

# Load the model once
model, tokenizer = load_fine_tuned_model()

# Use it multiple times
response = generate_response(
    model,
    tokenizer,
    question = "O que é plano de parentalidade",
    context = "O que é plano de parentalidade"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
print(generate_response(
    model,
    tokenizer,
    question = "O que é plano de parentalidade",
    context = "O que é plano de parentalidade"
))

<|im_start|> system
You are a helpful assistant that provides clear and informative answers based on the given context. If no context is provided or if it's marked as "NONE", provide a general answer based on your knowledge.
 <|im_end|> 
 <|im_start|> user
Context: O que é plano de parentalidade
Question: O que é plano de parentalidade
 <|im_end|> 
 <|im_start|> assistant

In Portugal, we have all heard about Plano Pessoa, but what does this mean? Is there anything special about this particular scheme of things in terms of inheritance law? The short version: yes. So far, the succession rules were very simple – children inherit their father’s fortune; sons get more than daughters do, etc., etc. However, these days it can be quite complicated to interpret when an inheriting child receives part of his/her inheritance before he or she turns 18 (called “the age limit”). This means some planning will need to take place before any funds are distributed among family members so they don't run a

.